In [20]:
%pip install pmdarima
%pip instal pmdarima

   ---------------------------------------- 0.0/719.3 kB ? eta -:--:--
   ---------------------------------------- 719.3/719.3 kB 14.7 MB/s  0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 2.8/2.8 MB 26.6 MB/s  0:00:00

   ---------------------------------------- 0/2 [Cython]
   ---------------------------------------- 0/2 [Cython]
   ---------------------------------------- 0/2 [Cython]
   ---------------------------------------- 0/2 [Cython]
   ---------------------------------------- 0/2 [Cython]
   ---------------------------------------- 0/2 [Cython]
   ---------------------------------------- 0/2 [Cython]
   ---------------------------------------- 0/2 [Cython]
   ---------------------------------------- 0/2 [Cython]
   ---------------------------------------- 0/2 [Cython]
   ---------------------------------------- 0/2 [Cython]
   ---------------------------------------- 0/2 [Cython]
   --------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "instal" - maybe you meant "install"



In [1]:
import numpy as np
import pandas as pd
import os 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm

## SEPARACION DE DATA

In [14]:
import pandas as pd
import os

file_routes = ['../data/data_conagua_clasificada/Seco/NR_25037.csv',
               '../data/data_conagua_clasificada/Templado/NR_25033.csv',
               '../data/data_conagua_clasificada/Tropical/NR_25046.csv']

output_dirs = ['./Sarimax_dataset_splits/Seco/NR_25037/',
               './Sarimax_dataset_splits/Templado/NR_25033/',
               './Sarimax_dataset_splits/Tropical/NR_25046/']

routes = [(file_route, output_dir) for file_route, output_dir in zip(file_routes, output_dirs)]

columnas_interes = ['EVAP', 'PRECIP', 'TMAX', 'TMIN'] 

for file_route, output_dir in routes:

    df = pd.read_csv(file_route, parse_dates=['FECHA'], index_col='FECHA')
    print(f"Datos cargados: {file_route}")

    df_continuo = df.asfreq('D')

    columnas_existentes = []
    for col in columnas_interes:
        if col in df.columns:
            df_continuo[f'mask_{col}'] = df_continuo[col].notna().astype(int)
            columnas_existentes.append(col)
        else:
            print(f"Advertencia: {col} no existe en este archivo.")


    n = len(df_continuo)
    train_end = int(n * 0.7)
    test_end = int(n * 0.9)

    train = df_continuo.iloc[:train_end]
    test = df_continuo.iloc[train_end:test_end]
    validation = df_continuo.iloc[test_end:]

    os.makedirs(os.path.join(output_dir, 'train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'test'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'validation'), exist_ok=True)

    print(f"Guardando archivos completos en {output_dir}...")
    train.to_csv(os.path.join(output_dir, 'train/train_completo.csv'))
    test.to_csv(os.path.join(output_dir, 'test/test_completo.csv'))
    validation.to_csv(os.path.join(output_dir, 'validation/validation_completo.csv'))


    for col in columnas_existentes:
        cols_to_save = [col, f'mask_{col}']
        
        train[cols_to_save].to_csv(os.path.join(output_dir, f'train/train_{col}.csv'))
        test[cols_to_save].to_csv(os.path.join(output_dir, f'test/test_{col}.csv'))
        validation[cols_to_save].to_csv(os.path.join(output_dir, f'validation/validation_{col}.csv'))

    print("Proceso finalizado para este archivo.\n")

Datos cargados: ../data/data_conagua_clasificada/Seco/NR_25037.csv
Guardando archivos completos en ./Sarimax_dataset_splits/Seco/NR_25037/...
Proceso finalizado para este archivo.

Datos cargados: ../data/data_conagua_clasificada/Templado/NR_25033.csv
Guardando archivos completos en ./Sarimax_dataset_splits/Templado/NR_25033/...
Proceso finalizado para este archivo.

Datos cargados: ../data/data_conagua_clasificada/Tropical/NR_25046.csv
Guardando archivos completos en ./Sarimax_dataset_splits/Tropical/NR_25046/...
Proceso finalizado para este archivo.



### Prueba de Estacionariedad Dickey-Fuller

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

file_dirs = ['./Sarimax_dataset_splits/Seco/NR_25037/train/train_completo.csv',
             './Sarimax_dataset_splits/Templado/NR_25033/train/train_completo.csv',
             './Sarimax_dataset_splits/Tropical/NR_25046/train/train_completo.csv']

default_output_dir = './data_analysis/Dickey-Fuller/'
os.makedirs(default_output_dir, exist_ok=True)

columnas_interes = ['EVAP', 'PRECIP', 'TMAX', 'TMIN']


for file_dir in file_dirs:
    df = pd.read_csv(file_dir, parse_dates=['FECHA'], index_col='FECHA')
    print(f"Datos cargados: {file_dir}")
    
    
    estacion = 'Desconocida'
    clasificacion = 'Desconocida'
    
    if 'ESTACIÓN' in df.columns:
        estacion = str(df['ESTACIÓN'].iloc[0])
        
    if 'CLASIFICACION' in df.columns:
        clasificacion = str(df['CLASIFICACION'].iloc[0])
        
        
    output_dir = os.path.join(default_output_dir, clasificacion, estacion)
    os.makedirs(output_dir, exist_ok=True)
    
    reporte_output_dir = os.path.join(output_dir, f'reporte_ADF.txt')
    #La prueba de Dickey-Fuller Aumentada (ADF) es una prueba estadística utilizada para determinar si una serie temporal es estacionaria o no.
    #ADF prueba la hipótesis nula de que una unidad raíz está presente en una serie temporal.
    #P valor bajo (generalmente ≤ 0.05) indica que se puede rechazar la hipótesis nula, sugiriendo que la serie temporal es estacionaria.
    #D = 0: Serie estacionaria
    #D >= 1: Serie no estacionaria, se recomienda diferenciar.
    
    with open(reporte_output_dir, 'w') as reporte_file:
        reporte_file.write(f"Reporte de Prueba de Dickey-Fuller Aumentada (ADF) para {estacion} - {clasificacion}\n")
        
        for col in columnas_interes:
            if col not in df.columns:
                
                
                print(f"Advertencia: {col} no existe en {file_dir}. Saltando análisis.")
                continue

            reporte_file.write(f"Análisis para {col}:\n")
            
            series = df[col].dropna()
            
            print(f"\nAnálisis para {col} en {file_dir}:")
            result = adfuller(series)
            
            
            # Estadística ADF 
            print(f'Estadistica ADF: {result[0]}')
            reporte_file.write(f'Estadistica ADF: {result[0]:.4f}\n\n')
            
            # Valor P
            print(f'Valor P: {result[1]}')        
            reporte_file.write(f'Valor P: {result[1]:.4f}\n\n')
            
            #Valores críticos
            for key, value in result[4].items():
                print(f'Valor Crítico ({key}): {value}')
                reporte_file.write(f'Valor Crítico ({key}): {value:.4f}\n')
            
            # Conclusión
            if result[1] < 0.05:
                valor_d = 0
                print("La serie es estacionaria. (d=0)")
                reporte_file.write("La serie es estacionaria. (d=0)\n\n")
            else:
                valor_d = 1
                print("La serie no es estacionaria. Considera diferenciar. (d>=1)")
                reporte_file.write("La serie no es estacionaria. Considera diferenciar. (d>=1)\n\n")
                
            reporte_file.write("\n")
            reporte_file.write(f"Resultado: Estadística ADF = {result[0]:.4f}, Valor P = {result[1]:.4f}, Valor D = {valor_d}\n")
                
                
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
            
            # ACF: Ayuda a identificar el término MA (q)
            plot_acf(series, ax=ax1, lags=40)
            ax1.set_title(f'Función de Autocorrelación (ACF) para {col} en {estacion} - {clasificacion}')
            
            # PACF: Ayuda a identificar el término AR (p)
            plot_pacf(series, ax=ax2, lags=40, method='ywm')
            ax2.set_title(f'Función de Autocorrelación Parcial (PACF) para {col} en {estacion} - {clasificacion}')
            
            plt.tight_layout()
            
            plt.savefig(os.path.join(output_dir, f'plot_ACF_PACF_{col}.png'))
            plt.close()

Datos cargados: ./Sarimax_dataset_splits/Seco/NR_25037/train/train_completo.csv

Análisis para EVAP en ./Sarimax_dataset_splits/Seco/NR_25037/train/train_completo.csv:
------------------------------
Estadistica ADF: -10.676035729060786
Valor P: 4.0516909305182395e-19
Valor Crítico (1%): -3.430743118305158
Valor Crítico (5%): -2.8617137425399806
Valor Crítico (10%): -2.566862478742594
La serie es estacionaria. (d=0)

Análisis para PRECIP en ./Sarimax_dataset_splits/Seco/NR_25037/train/train_completo.csv:
------------------------------
Estadistica ADF: -14.601906290872467
Valor P: 4.1529441129528554e-27
Valor Crítico (1%): -3.4307382864424487
Valor Crítico (5%): -2.8617116071898954
Valor Crítico (10%): -2.566861342125719
La serie es estacionaria. (d=0)

Análisis para TMAX en ./Sarimax_dataset_splits/Seco/NR_25037/train/train_completo.csv:
------------------------------
Estadistica ADF: -10.213647209319728
Valor P: 5.554001572168047e-18
Valor Crítico (1%): -3.4307380560249783
Valor Crític

## Busqueda Automatica de Hiperparametros Hyndman-Khandakar

### HiperParametros Evap

In [ ]:
import pandas as pd
import numpy as np
import pmdarima as pm
import os
import json

#Rutas Entradas y Salidas
rutas_procesamiento = [('./Sarimax_dataset_splits/Seco/NR_25037/train/train_completo.csv', './data_analysis/hyperparameters/Seco_NR_25037/'),
    ('./Sarimax_dataset_splits/Templado/NR_25033/train/train_completo.csv', './data_analysis/hyperparameters/Templado_NR_25033/'),
    ('./Sarimax_dataset_splits/Tropical/NR_25046/train/train_completo.csv', './data_analysis/hyperparameters/Tropical_NR_25046/')
]

#Busqueda de hiperparametros AutoARIMA con Fourier y Dynamic Harmonic Regression
#Hyndman - Khandakar postula una metodología para seleccionar automáticamente los parámetros del modelo ARIMA utilizando un enfoque basado en pruebas estadísticas y criterios de información.
#El procedimiento incluye:
#1. Identificación de la necesidad de diferenciación (d) mediante pruebas de raíz unitaria como la prueba de Dickey-Fuller aumentada.
#2. Selección inicial de los órdenes p y q basados en la inspección de las funciones de autocorrelación (ACF) y autocorrelación parcial (PACF).
#3. Búsqueda iterativa de los mejores parámetros p, d, q utilizando criterios de información como AIC o BIC para evaluar la calidad del modelo.
#4. Validación cruzada y diagnóstico de residuos para asegurar que el modelo seleccionado es adecuado para los datos.
    
#Definicion de columnas objetivo y exógenas
target_col = 'EVAP'
original_exog_cols = ['TMAX', 'TMIN', 'PRECIP']


for input_path, output_dir in rutas_procesamiento:
    os.makedirs(output_dir, exist_ok=True)

    df = pd.read_csv(input_path, parse_dates=['FECHA'], index_col='FECHA')
    
    try:
        df = pd.read_csv(input_path, parse_dates=['FECHA'], index_col='FECHA').asfreq('D')
    except Exception as e:
        print(f"Error al cargar {input_path}: {e}")
        continue
    
    #Generar Fourier con Curva anual suave
    K_Fourier = 2
    
    df_fourier = df.copy()
    
    t = np.arange(len(df_fourier)) +1
    
    for k in range(1, K_Fourier + 1):
        omega = 2 * np.pi * k / 365.25
        df_fourier[f'sin_{k}'] = np.sin(omega * t)
        df_fourier[f'cos_{k}'] = np.cos(omega * t)
    
    fourier_cols = [col for col in df_fourier.columns if 'sin_' in col or 'cos_' in col]
    
    
    #Combinar Exogenas con Fourier
    x_full = pd.concat([df_fourier[original_exog_cols], df_fourier[fourier_cols]], axis=1)
    
    combined = pd.concat([df_fourier[target_col], x_full], axis=1).dropna()
    
    y_clean = combined[target_col]
    X_clean = combined.drop(columns=[target_col])
    
    
    Estacion = 'Desconocida'
    Clasificacion = 'Desconocida'
    if 'ESTACIÓN' in df.columns:
        Estacion = str(df['ESTACIÓN'].iloc[0])
    if 'CLASIFICACION' in df.columns:
        Clasificacion = str(df['CLASIFICACION'].iloc[0])
        
    print(f'Iniciando AutoARIMA para Estación: {Estacion}, Clasificación: {Clasificacion}')
    print('Variables Exógenas utilizadas:', X_clean.columns.tolist())
    
    model_auto = pm.auto_arima(y_clean,
                               exogenous=X_clean,
                               start_p=0,
                               start_q=0,
                               max_p=5,
                               max_q=3,
                               d=0,
                               seasonal=False, #Se quita estacionalidad por presencia de Fourier
                               trace=False,
                               error_action='ignore',
                               suppress_warnings=True,
                               stepwise=True)
    
    resultados = {
        "Estacion": Estacion,
        "metodo": "Dynamic Harmonic Regression con AutoARIMA",
        "Clasificacion": Clasificacion,
        "fourier_K" : K_Fourier,
        "exogenous_variables": X_clean.columns.tolist(),
        "best_arima_order": model_auto.order,
        "AIC": model_auto.aic(),
        "BIC": model_auto.bic()
    }
    
    output_json_path = os.path.join(output_dir, f'best_model_params_{target_col}.json')
    with open(output_json_path, 'w') as json_file:
        json.dump(resultados, json_file, indent=4)
        
    print(f'Modelo guardado en {output_json_path}\n')
    print("Orden Arima Encontrada:", model_auto.order)
    
    
    


Iniciando AutoARIMA para Estación: 25037.0, Clasificación: Seco
Variables Exógenas utilizadas: ['TMAX', 'TMIN', 'PRECIP', 'sin_1', 'cos_1', 'sin_2', 'cos_2']
Modelo guardado en ./data_analysis/hyperparameters/Seco_NR_25037/best_model_params_EVAP.json

Orden Arima Encontrada: (4, 0, 3)
Iniciando AutoARIMA para Estación: 25033.0, Clasificación: Templado
Variables Exógenas utilizadas: ['TMAX', 'TMIN', 'PRECIP', 'sin_1', 'cos_1', 'sin_2', 'cos_2']
Modelo guardado en ./data_analysis/hyperparameters/Templado_NR_25033/best_model_params_EVAP.json

Orden Arima Encontrada: (3, 0, 1)
Iniciando AutoARIMA para Estación: 25046.0, Clasificación: Tropical
Variables Exógenas utilizadas: ['TMAX', 'TMIN', 'PRECIP', 'sin_1', 'cos_1', 'sin_2', 'cos_2']
Modelo guardado en ./data_analysis/hyperparameters/Tropical_NR_25046/best_model_params_EVAP.json

Orden Arima Encontrada: (3, 0, 1)


### Hiperparametros Precip

In [26]:
import pandas as pd
import numpy as np
import pmdarima as pm
import os
import json

#Rutas Entradas y Salidas
rutas_procesamiento = [('./Sarimax_dataset_splits/Seco/NR_25037/train/train_completo.csv', './data_analysis/hyperparameters/Seco_NR_25037/'),
    ('./Sarimax_dataset_splits/Templado/NR_25033/train/train_completo.csv', './data_analysis/hyperparameters/Templado_NR_25033/'),
    ('./Sarimax_dataset_splits/Tropical/NR_25046/train/train_completo.csv', './data_analysis/hyperparameters/Tropical_NR_25046/')
]


    
#Definicion de columnas objetivo y exógenas
target_col = 'PRECIP'
original_exog_cols = ['EVAP', 'TMAX', 'TMIN']


for input_path, output_dir in rutas_procesamiento:
    os.makedirs(output_dir, exist_ok=True)

    df = pd.read_csv(input_path, parse_dates=['FECHA'], index_col='FECHA')
    
    try:
        df = pd.read_csv(input_path, parse_dates=['FECHA'], index_col='FECHA').asfreq('D')
    except Exception as e:
        print(f"Error al cargar {input_path}: {e}")
        continue
    
    #Generar Fourier con Curva anual suave
    K_Fourier = 2
    
    df_fourier = df.copy()
    
    t = np.arange(len(df_fourier)) +1
    
    for k in range(1, K_Fourier + 1):
        omega = 2 * np.pi * k / 365.25
        df_fourier[f'sin_{k}'] = np.sin(omega * t)
        df_fourier[f'cos_{k}'] = np.cos(omega * t)
    
    fourier_cols = [col for col in df_fourier.columns if 'sin_' in col or 'cos_' in col]
    
    
    #Combinar Exogenas con Fourier
    x_full = pd.concat([df_fourier[original_exog_cols], df_fourier[fourier_cols]], axis=1)
    
    combined = pd.concat([df_fourier[target_col], x_full], axis=1).dropna()
    
    y_clean = combined[target_col]
    X_clean = combined.drop(columns=[target_col])
    
    
    Estacion = 'Desconocida'
    Clasificacion = 'Desconocida'
    if 'ESTACIÓN' in df.columns:
        Estacion = str(df['ESTACIÓN'].iloc[0])
    if 'CLASIFICACION' in df.columns:
        Clasificacion = str(df['CLASIFICACION'].iloc[0])
        
    print(f'Iniciando AutoARIMA para Estación: {Estacion}, Clasificación: {Clasificacion}')
    print('Variables Exógenas utilizadas:', X_clean.columns.tolist())
    
    model_auto = pm.auto_arima(y_clean,
                               exogenous=X_clean,
                               start_p=0,
                               start_q=0,
                               max_p=5,
                               max_q=3,
                               d=0,
                               seasonal=False, #Se quita estacionalidad por presencia de Fourier
                               trace=False,
                               error_action='ignore',
                               suppress_warnings=True,
                               stepwise=True)
    
    resultados = {
        "Estacion": Estacion,
        "metodo": "Dynamic Harmonic Regression con AutoARIMA",
        "Clasificacion": Clasificacion,
        "fourier_K" : K_Fourier,
        "exogenous_variables": X_clean.columns.tolist(),
        "best_arima_order": model_auto.order,
        "AIC": model_auto.aic(),
        "BIC": model_auto.bic()
    }
    
    output_json_path = os.path.join(output_dir, f'best_model_params_{target_col}.json')
    with open(output_json_path, 'w') as json_file:
        json.dump(resultados, json_file, indent=4)
        
    print(f'Modelo guardado en {output_json_path}\n')
    print("Orden Arima Encontrada:", model_auto.order)
    
    
    


Iniciando AutoARIMA para Estación: 25037.0, Clasificación: Seco
Variables Exógenas utilizadas: ['EVAP', 'TMAX', 'TMIN', 'sin_1', 'cos_1', 'sin_2', 'cos_2']
Modelo guardado en ./data_analysis/hyperparameters/Seco_NR_25037/best_model_params_PRECIP.json

Orden Arima Encontrada: (5, 0, 1)
Iniciando AutoARIMA para Estación: 25033.0, Clasificación: Templado
Variables Exógenas utilizadas: ['EVAP', 'TMAX', 'TMIN', 'sin_1', 'cos_1', 'sin_2', 'cos_2']
Modelo guardado en ./data_analysis/hyperparameters/Templado_NR_25033/best_model_params_PRECIP.json

Orden Arima Encontrada: (5, 0, 1)
Iniciando AutoARIMA para Estación: 25046.0, Clasificación: Tropical
Variables Exógenas utilizadas: ['EVAP', 'TMAX', 'TMIN', 'sin_1', 'cos_1', 'sin_2', 'cos_2']
Modelo guardado en ./data_analysis/hyperparameters/Tropical_NR_25046/best_model_params_PRECIP.json

Orden Arima Encontrada: (1, 0, 2)


### Hiperparametros TMAX

In [27]:
import pandas as pd
import numpy as np
import pmdarima as pm
import os
import json

#Rutas Entradas y Salidas
rutas_procesamiento = [('./Sarimax_dataset_splits/Seco/NR_25037/train/train_completo.csv', './data_analysis/hyperparameters/Seco_NR_25037/'),
    ('./Sarimax_dataset_splits/Templado/NR_25033/train/train_completo.csv', './data_analysis/hyperparameters/Templado_NR_25033/'),
    ('./Sarimax_dataset_splits/Tropical/NR_25046/train/train_completo.csv', './data_analysis/hyperparameters/Tropical_NR_25046/')
]


    
#Definicion de columnas objetivo y exógenas
target_col = 'TMAX'
original_exog_cols = ['EVAP', 'PRECIP', 'TMIN']


for input_path, output_dir in rutas_procesamiento:
    os.makedirs(output_dir, exist_ok=True)

    df = pd.read_csv(input_path, parse_dates=['FECHA'], index_col='FECHA')
    
    try:
        df = pd.read_csv(input_path, parse_dates=['FECHA'], index_col='FECHA').asfreq('D')
    except Exception as e:
        print(f"Error al cargar {input_path}: {e}")
        continue
    
    #Generar Fourier con Curva anual suave
    K_Fourier = 2
    
    df_fourier = df.copy()
    
    t = np.arange(len(df_fourier)) +1
    
    for k in range(1, K_Fourier + 1):
        omega = 2 * np.pi * k / 365.25
        df_fourier[f'sin_{k}'] = np.sin(omega * t)
        df_fourier[f'cos_{k}'] = np.cos(omega * t)
    
    fourier_cols = [col for col in df_fourier.columns if 'sin_' in col or 'cos_' in col]
    
    
    #Combinar Exogenas con Fourier
    x_full = pd.concat([df_fourier[original_exog_cols], df_fourier[fourier_cols]], axis=1)
    
    combined = pd.concat([df_fourier[target_col], x_full], axis=1).dropna()
    
    y_clean = combined[target_col]
    X_clean = combined.drop(columns=[target_col])
    
    
    Estacion = 'Desconocida'
    Clasificacion = 'Desconocida'
    if 'ESTACIÓN' in df.columns:
        Estacion = str(df['ESTACIÓN'].iloc[0])
    if 'CLASIFICACION' in df.columns:
        Clasificacion = str(df['CLASIFICACION'].iloc[0])
        
    print(f'Iniciando AutoARIMA para Estación: {Estacion}, Clasificación: {Clasificacion}')
    print('Variables Exógenas utilizadas:', X_clean.columns.tolist())
    
    model_auto = pm.auto_arima(y_clean,
                               exogenous=X_clean,
                               start_p=0,
                               start_q=0,
                               max_p=5,
                               max_q=3,
                               d=0,
                               seasonal=False, #Se quita estacionalidad por presencia de Fourier
                               trace=False,
                               error_action='ignore',
                               suppress_warnings=True,
                               stepwise=True)
    
    resultados = {
        "Estacion": Estacion,
        "metodo": "Dynamic Harmonic Regression con AutoARIMA",
        "Clasificacion": Clasificacion,
        "fourier_K" : K_Fourier,
        "exogenous_variables": X_clean.columns.tolist(),
        "best_arima_order": model_auto.order,
        "AIC": model_auto.aic(),
        "BIC": model_auto.bic()
    }
    
    output_json_path = os.path.join(output_dir, f'best_model_params_{target_col}.json')
    with open(output_json_path, 'w') as json_file:
        json.dump(resultados, json_file, indent=4)
        
    print(f'Modelo guardado en {output_json_path}\n')
    print("Orden Arima Encontrada:", model_auto.order)
    
    
    


Iniciando AutoARIMA para Estación: 25037.0, Clasificación: Seco
Variables Exógenas utilizadas: ['EVAP', 'PRECIP', 'TMIN', 'sin_1', 'cos_1', 'sin_2', 'cos_2']
Modelo guardado en ./data_analysis/hyperparameters/Seco_NR_25037/best_model_params_TMAX.json

Orden Arima Encontrada: (4, 0, 3)
Iniciando AutoARIMA para Estación: 25033.0, Clasificación: Templado
Variables Exógenas utilizadas: ['EVAP', 'PRECIP', 'TMIN', 'sin_1', 'cos_1', 'sin_2', 'cos_2']
Modelo guardado en ./data_analysis/hyperparameters/Templado_NR_25033/best_model_params_TMAX.json

Orden Arima Encontrada: (3, 0, 1)
Iniciando AutoARIMA para Estación: 25046.0, Clasificación: Tropical
Variables Exógenas utilizadas: ['EVAP', 'PRECIP', 'TMIN', 'sin_1', 'cos_1', 'sin_2', 'cos_2']
Modelo guardado en ./data_analysis/hyperparameters/Tropical_NR_25046/best_model_params_TMAX.json

Orden Arima Encontrada: (3, 0, 3)


### Hiperparametros TMIN

In [28]:
import pandas as pd
import numpy as np
import pmdarima as pm
import os
import json

#Rutas Entradas y Salidas
rutas_procesamiento = [('./Sarimax_dataset_splits/Seco/NR_25037/train/train_completo.csv', './data_analysis/hyperparameters/Seco_NR_25037/'),
    ('./Sarimax_dataset_splits/Templado/NR_25033/train/train_completo.csv', './data_analysis/hyperparameters/Templado_NR_25033/'),
    ('./Sarimax_dataset_splits/Tropical/NR_25046/train/train_completo.csv', './data_analysis/hyperparameters/Tropical_NR_25046/')
]


    
#Definicion de columnas objetivo y exógenas
target_col = 'TMIN'
original_exog_cols = ['EVAP', 'PRECIP', 'TMAX']


for input_path, output_dir in rutas_procesamiento:
    os.makedirs(output_dir, exist_ok=True)

    df = pd.read_csv(input_path, parse_dates=['FECHA'], index_col='FECHA')
    
    try:
        df = pd.read_csv(input_path, parse_dates=['FECHA'], index_col='FECHA').asfreq('D')
    except Exception as e:
        print(f"Error al cargar {input_path}: {e}")
        continue
    
    #Generar Fourier con Curva anual suave
    K_Fourier = 2
    
    df_fourier = df.copy()
    
    t = np.arange(len(df_fourier)) +1
    
    for k in range(1, K_Fourier + 1):
        omega = 2 * np.pi * k / 365.25
        df_fourier[f'sin_{k}'] = np.sin(omega * t)
        df_fourier[f'cos_{k}'] = np.cos(omega * t)
    
    fourier_cols = [col for col in df_fourier.columns if 'sin_' in col or 'cos_' in col]
    
    
    #Combinar Exogenas con Fourier
    x_full = pd.concat([df_fourier[original_exog_cols], df_fourier[fourier_cols]], axis=1)
    
    combined = pd.concat([df_fourier[target_col], x_full], axis=1).dropna()
    
    y_clean = combined[target_col]
    X_clean = combined.drop(columns=[target_col])
    
    
    Estacion = 'Desconocida'
    Clasificacion = 'Desconocida'
    if 'ESTACIÓN' in df.columns:
        Estacion = str(df['ESTACIÓN'].iloc[0])
    if 'CLASIFICACION' in df.columns:
        Clasificacion = str(df['CLASIFICACION'].iloc[0])
        
    print(f'Iniciando AutoARIMA para Estación: {Estacion}, Clasificación: {Clasificacion}')
    print('Variables Exógenas utilizadas:', X_clean.columns.tolist())
    
    model_auto = pm.auto_arima(y_clean,
                               exogenous=X_clean,
                               start_p=0,
                               start_q=0,
                               max_p=5,
                               max_q=3,
                               d=0,
                               seasonal=False, #Se quita estacionalidad por presencia de Fourier
                               trace=False,
                               error_action='ignore',
                               suppress_warnings=True,
                               stepwise=True)
    
    resultados = {
        "Estacion": Estacion,
        "metodo": "Dynamic Harmonic Regression con AutoARIMA",
        "Clasificacion": Clasificacion,
        "fourier_K" : K_Fourier,
        "exogenous_variables": X_clean.columns.tolist(),
        "best_arima_order": model_auto.order,
        "AIC": model_auto.aic(),
        "BIC": model_auto.bic()
    }
    
    output_json_path = os.path.join(output_dir, f'best_model_params_{target_col}.json')
    with open(output_json_path, 'w') as json_file:
        json.dump(resultados, json_file, indent=4)
        
    print(f'Modelo guardado en {output_json_path}\n')
    print("Orden Arima Encontrada:", model_auto.order)
    
    
    


Iniciando AutoARIMA para Estación: 25037.0, Clasificación: Seco
Variables Exógenas utilizadas: ['EVAP', 'PRECIP', 'TMAX', 'sin_1', 'cos_1', 'sin_2', 'cos_2']
Modelo guardado en ./data_analysis/hyperparameters/Seco_NR_25037/best_model_params_TMIN.json

Orden Arima Encontrada: (4, 0, 1)
Iniciando AutoARIMA para Estación: 25033.0, Clasificación: Templado
Variables Exógenas utilizadas: ['EVAP', 'PRECIP', 'TMAX', 'sin_1', 'cos_1', 'sin_2', 'cos_2']
Modelo guardado en ./data_analysis/hyperparameters/Templado_NR_25033/best_model_params_TMIN.json

Orden Arima Encontrada: (2, 0, 1)
Iniciando AutoARIMA para Estación: 25046.0, Clasificación: Tropical
Variables Exógenas utilizadas: ['EVAP', 'PRECIP', 'TMAX', 'sin_1', 'cos_1', 'sin_2', 'cos_2']
Modelo guardado en ./data_analysis/hyperparameters/Tropical_NR_25046/best_model_params_TMIN.json

Orden Arima Encontrada: (5, 0, 1)


## Entrenamiento de modelo

### Relleno de datos de entrenamiento con estaciones cercanas

In [ ]:
import pandas as pd
import numpy as np
import os
from math import radians, cos, sin, asin, sqrt


rutas_procesamiento = [
    ('./Sarimax_dataset_splits/Seco/NR_25037/train/train_completo.csv', '../data/data_conagua_clasificada/Seco'),
    ('./Sarimax_dataset_splits/Templado/NR_25033/train/train_completo.csv', '../data/data_conagua_clasificada/Templado'),
    ('./Sarimax_dataset_splits/Tropical/NR_25046/train/train_completo.csv', '../data/data_conagua_clasificada/Tropical')
]

MAX_DIST_KM = 50.0     
MAX_ALT_DIFF_M = 500.0 


for target_path, neighbors_dir in rutas_procesamiento:
    
    print(f"\n==================================================")
    print(f"Procesando Objetivo: {target_path}")
    print(f"Buscando ayuda en: {neighbors_dir}")
    
    # 1. CARGAR DATOS DEL OBJETIVO (TARGET)
    try:
        df_target = pd.read_csv(target_path, parse_dates=['FECHA'], index_col='FECHA').asfreq('D')
    except Exception as e:
        print(f"Error cargando el archivo objetivo: {e}")
        continue

    try:
        target_meta = df_target.iloc[0]
        lat_t = float(target_meta['LATITUD'])
        lon_t = float(target_meta['LONGITUD'])
        alt_t = float(target_meta['ALTITUD'])
        code_t = str(target_meta['ESTACIÓN'])
    except:
        print("No se pudieron leer los metadatos .")
        continue


    cols_interes = ['PRECIP', 'TMAX', 'TMIN', 'EVAP']
    cols_existentes = [c for c in cols_interes if c in df_target.columns]
    
    nans_iniciales = df_target[cols_existentes].isna().sum().sum()
    print(f"Huecos iniciales a llenar: {nans_iniciales}")
    
    if nans_iniciales == 0:
        print("-> Archivo completo. No requiere imputación.")
        continue

    # ESCANEAR CARPETA DE VECINOS (Discovery)
    posibles_vecinos = [f for f in os.listdir(neighbors_dir) if f.endswith('.csv')]
    vecinos_validos = []
    for archivo_vecino in posibles_vecinos:
        path_vecino = os.path.join(neighbors_dir, archivo_vecino)
        if str(code_t) in archivo_vecino: 
            continue 

        try:
            df_temp = pd.read_csv(path_vecino, nrows=5)
            
            if 'LATITUD' not in df_temp.columns: continue

            lat_v = float(df_temp['LATITUD'].iloc[0])
            lon_v = float(df_temp['LONGITUD'].iloc[0])
            alt_v = float(df_temp['ALTITUD'].iloc[0])
            code_v = str(df_temp['CODIGO'].iloc[0])

            # Haversine inline
            r_lat_t, r_lon_t = radians(lat_t), radians(lon_t)
            r_lat_v, r_lon_v = radians(lat_v), radians(lon_v)
            
            dlon = r_lon_v - r_lon_t
            dlat = r_lat_v - r_lat_t
            
            a = sin(dlat/2)**2 + cos(r_lat_t) * cos(r_lat_v) * sin(dlon/2)**2
            c = 2 * asin(sqrt(a))
            dist_km = 6371 * c # Radio tierra

            #Diferencia de altura
            alt_diff = abs(alt_t - alt_v)

            #Filtrar por criterios
            if dist_km <= MAX_DIST_KM and alt_diff <= MAX_ALT_DIFF_M:
                vecinos_validos.append({
                    'path': path_vecino,
                    'ESTACIÓN': code_v,
                    'distancia': dist_km,
                    'alt_diff': alt_diff
                })

        except Exception as e:
            continue 


    vecinos_validos.sort(key=lambda x: x['distancia'])
    
    print(f"Vecinos útiles encontrados: {len(vecinos_validos)}")
    
    if len(vecinos_validos) > 0:
        print("   Iniciando transfusión de datos...")
        
        for vecino in vecinos_validos:
            # Verificamos si aun faltan datos
            if df_target[cols_existentes].isna().sum().sum() == 0:
                break
                
            try:
                df_donador = pd.read_csv(vecino['path'], parse_dates=['FECHA'], index_col='FECHA').asfreq('D')
                
                print(f"     -> Tomando datos de: {vecino['ESTACIÓN']} (Dist: {vecino['distancia']:.1f}km)")
                
                for col in cols_existentes:
                    if col in df_donador.columns:
                        df_target[col] = df_target[col].fillna(df_donador[col])
                        
            except:
                continue

        nans_finales = df_target[cols_existentes].isna().sum().sum()
        recuperados = nans_iniciales - nans_finales
        
        if recuperados > 0:
            print(f"IMPUTACIÓN EXITOSA: Se recuperaron {recuperados} registros.")
            df_target.to_csv(target_path)
            print(f"      Archivo actualizado: {target_path}")
        else:
            print("Los vecinos encontrados no tenían datos para las fechas faltantes.")
            
    else:
        print("No se encontraron vecinos que cumplan los criterios de Distancia/Altitud.")


--- INICIANDO IMPUTACIÓN ESPACIAL CON DIRECTORIOS DE APOYO ---

📂 Procesando Objetivo: ./Sarimax_dataset_splits/Seco/NR_25037/train/train_completo.csv
🔎 Buscando ayuda en: ../data/data_conagua_clasificada/Seco
   Huecos iniciales a llenar: 6925
   Vecinos útiles encontrados: 0
   ⚠️ No se encontraron vecinos que cumplan los criterios de Distancia/Altitud.

📂 Procesando Objetivo: ./Sarimax_dataset_splits/Templado/NR_25033/train/train_completo.csv
🔎 Buscando ayuda en: ../data/data_conagua_clasificada/Templado
   Huecos iniciales a llenar: 1515
   Vecinos útiles encontrados: 0
   ⚠️ No se encontraron vecinos que cumplan los criterios de Distancia/Altitud.

📂 Procesando Objetivo: ./Sarimax_dataset_splits/Tropical/NR_25046/train/train_completo.csv
🔎 Buscando ayuda en: ../data/data_conagua_clasificada/Tropical
   Huecos iniciales a llenar: 3873
   Vecinos útiles encontrados: 0
   ⚠️ No se encontraron vecinos que cumplan los criterios de Distancia/Altitud.

--- PROCESO TERMINADO ---


## Entrenamiento SARIMAX

In [ ]:
import pandas as pd
import statsmodels.api as sm
import json
import joblib
import numpy as np
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

# Ignorar warnings
warnings.simplefilter('ignore', ConvergenceWarning)

casos_a_recalibrar = [
    {
        "region": "Seco",
        "station": "25037",
        "target": "TMAX",
        "data_path": "./Sarimax_dataset_splits/Seco/NR_25037/train/train_completo.csv",
        "json_path": "./data_analysis/hyperparameters/Seco_NR_25037/best_model_params_TMAX.json",
        "model_out": "./models/trained_models/SARIMAX/SARIMAX_25037.0_Seco_TMAX.pkl"
    },
    {
        "region": "Tropical",
        "station": "25046",
        "target": "TMAX",
        "data_path": "./Sarimax_dataset_splits/Tropical/NR_25046/train/train_completo.csv",
        "json_path": "./data_analysis/hyperparameters/Tropical_NR_25046/best_model_params_TMAX.json",
        "model_out": "./models/trained_models/SARIMAX/SARIMAX_25046.0_Tropical_TMAX.pkl"
    },
    {
        "region": "Tropical",
        "station": "25046",
        "target": "TMIN",
        "data_path": "./Sarimax_dataset_splits/Tropical/NR_25046/train/train_completo.csv",
        "json_path": "./data_analysis/hyperparameters/Tropical_NR_25046/best_model_params_TMIN.json",
        "model_out": "./models/trained_models/SARIMAX/SARIMAX_25046.0_Tropical_TMIN.pkl"
    }
]

print("--- INICIANDO RECALIBRACIÓN (TIPO NUMÉRICO FORZADO) ---\n")

for caso in casos_a_recalibrar:
    try:
        print(f"Procesando: {caso['region']} - {caso['station']} - {caso['target']}...")
        
        # 1. Cargar Datos
        df = pd.read_csv(caso['data_path'], index_col=0, parse_dates=True)
        df = df.asfreq('D')
        
        # 2. Cargar Hiperparámetros para saber qué columnas usar
        with open(caso['json_path'], 'r') as f:
            params = json.load(f)
        
        order = tuple(params['best_arima_order'])
        seasonal_order = tuple(params.get('seasonal_order', (0, 0, 0, 0)))
        
        
        # A. Seleccionar Target
        y_raw = df[caso['target']]
        # Forzar a numérico 
        y = pd.to_numeric(y_raw, errors='coerce').astype(float)
        
        # B. Seleccionar variables Exogenas
        cols_exogenas = params.get('exogenous_variables', [])
        
        if not cols_exogenas:
            X_raw = df.drop(columns=[caso['target']])
        else:

            cols_validas = [c for c in cols_exogenas if c in df.columns]
            X_raw = df[cols_validas]

        # Forzar todo X a numérico
        X = X_raw.apply(pd.to_numeric, errors='coerce').astype(float)
        
        # C. Limpieza final de NaNs 
        if y.isnull().any() or X.isnull().any().any():
            print("   -> Aviso: Se encontraron NaNs tras la conversión. Rellenando...")
            y = y.fillna(method='ffill').fillna(method='bfill')
            X = X.fillna(method='ffill').fillna(method='bfill')

        print(f"   -> Tipos de datos corregidos. X shape: {X.shape}")
        
        # 3. Definir Modelo
        model = sm.tsa.SARIMAX(
            endog=y,
            exog=X,
            order=order,
            seasonal_order=seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        
        # 4. Entrenar
        model_fit = model.fit(disp=False, maxiter=600, method='lbfgs')
        
        if model_fit.mle_retvals['converged']:
            estado = "CONVERGENCIA EXITOSA"
        else:
            print("   -> Advertencia: lbfgs no convergió. Intentando 'powell'...")
            model_fit = model.fit(disp=False, maxiter=1000, method='powell')
            estado = "SEGUNDO INTENTO FINALIZADO"
            
        print(f"   -> Resultado: {estado}")
        
        # 5. Guardar
        joblib.dump(model_fit, caso['model_out'])
        print(f"   -> Guardado en: {caso['model_out']}\n")

    except Exception as e:
        import traceback
        print(f"   -> ERROR CRÍTICO: {str(e)}")


--- INICIANDO RECALIBRACIÓN (TIPO NUMÉRICO FORZADO) ---

Procesando: Seco - 25037 - TMAX...
   -> Aviso: Se encontraron NaNs tras la conversión. Rellenando...
   -> Tipos de datos corregidos. X shape: (18557, 3)


C:\Users\Sarah\AppData\Local\Temp\ipykernel_30440\3361428809.py:84: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  y = y.fillna(method='ffill').fillna(method='bfill')
C:\Users\Sarah\AppData\Local\Temp\ipykernel_30440\3361428809.py:85: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = X.fillna(method='ffill').fillna(method='bfill')


   -> Resultado: CONVERGENCIA EXITOSA
   -> Guardado en: ./models/trained_models/SARIMAX/SARIMAX_25037.0_Seco_TMAX.pkl

Procesando: Tropical - 25046 - TMAX...
   -> Aviso: Se encontraron NaNs tras la conversión. Rellenando...


C:\Users\Sarah\AppData\Local\Temp\ipykernel_30440\3361428809.py:84: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  y = y.fillna(method='ffill').fillna(method='bfill')
C:\Users\Sarah\AppData\Local\Temp\ipykernel_30440\3361428809.py:85: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = X.fillna(method='ffill').fillna(method='bfill')


   -> Tipos de datos corregidos. X shape: (21391, 3)
   -> Resultado: CONVERGENCIA EXITOSA
   -> Guardado en: ./models/trained_models/SARIMAX/SARIMAX_25046.0_Tropical_TMAX.pkl

Procesando: Tropical - 25046 - TMIN...
   -> Aviso: Se encontraron NaNs tras la conversión. Rellenando...
   -> Tipos de datos corregidos. X shape: (21391, 3)


C:\Users\Sarah\AppData\Local\Temp\ipykernel_30440\3361428809.py:84: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  y = y.fillna(method='ffill').fillna(method='bfill')
C:\Users\Sarah\AppData\Local\Temp\ipykernel_30440\3361428809.py:85: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = X.fillna(method='ffill').fillna(method='bfill')


   -> Resultado: CONVERGENCIA EXITOSA
   -> Guardado en: ./models/trained_models/SARIMAX/SARIMAX_25046.0_Tropical_TMIN.pkl

--- FIN ---


## Testing del modelo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os
import warnings

warnings.simplefilter('ignore')

casos_evaluacion = [
    # SECO (25037)
    {"region": "Seco", "station": "25037", "target": "TMAX"},
    {"region": "Seco", "station": "25037", "target": "TMIN"},
    {"region": "Seco", "station": "25037", "target": "PRECIP"},
    {"region": "Seco", "station": "25037", "target": "EVAP"},
    # TROPICAL (25046)
    {"region": "Tropical", "station": "25046", "target": "TMAX"},
    {"region": "Tropical", "station": "25046", "target": "TMIN"},
    {"region": "Tropical", "station": "25046", "target": "PRECIP"},
    {"region": "Tropical", "station": "25046", "target": "EVAP"},
    # TEMPLADO (25033)
    {"region": "Templado", "station": "25033", "target": "TMAX"},
    {"region": "Templado", "station": "25033", "target": "TMIN"},
    {"region": "Templado", "station": "25033", "target": "PRECIP"},
    {"region": "Templado", "station": "25033", "target": "EVAP"},
]

# Definición de rangos (Bins)
bins = [0, 3, 7, 31, 180, 365, 730, float('inf')]
labels = ['1-3 Dias', '4-7 Dias', '8-31 Dias', '32-180 Dias', '181-365 Dias', '1-2 Años', '>2 Años']

print("--- GENERANDO REPORTES VISUALES LINEALES (SIN FUNCIONES) ---\n")

for caso in casos_evaluacion:
    try:
        region = caso['region']
        station = caso['station']
        target = caso['target']
        
        # Rutas
        test_path = f"./Sarimax_dataset_splits/{region}/NR_{station}/test/test_completo.csv"
        model_path = f"./models/trained_models/SARIMAX/SARIMAX_{station}.0_{region}_{target}.pkl"
        output_route = f"./data_analysis/Performance_metrics_plots/SARIMAX/{region}/NR_{station}/{target}/test"
        os.makedirs(output_route, exist_ok=True)

        if not os.path.exists(model_path):
            print(f"[SALTAR] Modelo no encontrado: {target} - {region}")
            continue

        print(f"Procesando: {region} - {target}...")
        df_test = pd.read_csv(test_path, index_col=0, parse_dates=True).asfreq('D')
        df_test = df_test.apply(pd.to_numeric, errors='coerce').ffill().bfill()
        model_fit = joblib.load(model_path)
        
        # Generación de Fourier 
        exog_needed = [x for x in model_fit.model.exog_names if x != 'const']
        
        if any('sin_' in col for col in exog_needed):
            day_of_year = df_test.index.dayofyear
            for k in [1, 2]:
                df_test[f'sin_{k}'] = np.sin(2 * np.pi * k * day_of_year / 365.25)
                df_test[f'cos_{k}'] = np.cos(2 * np.pi * k * day_of_year / 365.25)

        if len(exog_needed) > 0:
            for c in exog_needed:
                if c not in df_test.columns: df_test[c] = 0
            X_test = df_test[exog_needed]
        else:
            X_test = None

        # Predicción
        pred_result = model_fit.get_prediction(start=df_test.index[0], end=df_test.index[-1], exog=X_test)
        
        # Preparar DataFrame Base
        df_eval = pd.DataFrame({
            'y': df_test[target],
            'yhat': pred_result.predicted_mean
        })
        
        # Calcular Horizonte y Rangos
        df_eval['horizon_days'] = np.arange(1, len(df_eval) + 1)
        df_eval['range'] = pd.cut(df_eval['horizon_days'], bins=bins, labels=labels)
        
        # Iteramos manualmente por los labels para llenar la lista
        metrics_list = []
        
        for label in labels:
            # Filtramos el pedazo de datos correspondiente a este rango
            subset = df_eval[df_eval['range'] == label]
            
            if len(subset) > 0:
                y_true = subset['y'].values
                y_pred = subset['yhat'].values
                
                # RMSE / MSE
                mse = np.mean((y_true - y_pred) ** 2)
                rmse = np.sqrt(mse)
                
                # MAE
                mae = np.mean(np.abs(y_true - y_pred))
                
                # MAPE (Manejo manual de ceros)
                mask_nonzero = y_true != 0
                if np.sum(mask_nonzero) > 0:
                    mape = np.mean(np.abs((y_true[mask_nonzero] - y_pred[mask_nonzero]) / y_true[mask_nonzero]))
                else:
                    mape = 0.0
                
                count = len(y_true)
            else:
                mse, rmse, mae, mape, count = 0, 0, 0, 0, 0
            
            # Agregamos a la lista
            metrics_list.append({
                'range': label,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae,
                'MAPE': mape,
                'Count': count
            })
            
        # Convertimos la lista a DataFrame
        df_grouped_metrics = pd.DataFrame(metrics_list)

        #Generacion de reportes visuales
        display_df = df_grouped_metrics.copy()
        # Formateo de strings para visualización
        for c in ['MSE', 'RMSE', 'MAE', 'MAPE']:
            display_df[c] = display_df[c].apply(lambda x: f"{x:.4f}" if pd.notnull(x) else "-")
            
        rows, cols = display_df.shape
        fig_w = max(8, min(24, 1.5 * cols))
        fig_h = max(2, 0.5 * rows)
        
        fig, ax = plt.subplots(figsize=(fig_w, fig_h))
        ax.axis('off')
        
        table = ax.table(cellText=display_df.values, colLabels=display_df.columns, loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 1.5)
        
        png_table = os.path.join(output_route, f"metrics_table.png")
        fig.tight_layout()
        fig.savefig(png_table, dpi=150, bbox_inches='tight')
        plt.close(fig)
        print(f"   -> Tabla guardada")

        plt.figure(figsize=(10, 6))
        # Filtramos ceros para que no grafique barras vacías
        plot_data = df_grouped_metrics[df_grouped_metrics['Count'] > 0]
        
        bars = plt.bar(plot_data['range'], plot_data['RMSE'], color='skyblue', edgecolor='black')
        
        plt.title(f'RMSE por Horizonte - {region} {target}')
        plt.xlabel('Rango de Dias (Horizonte)')
        plt.ylabel('RMSE')
        plt.grid(axis='y', linestyle='--', alpha=0.7)

        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                     f'{height:.2f}',
                     ha='center', va='bottom')

        png_plot = os.path.join(output_route, f"RMSE_bar_plot.png")
        plt.savefig(png_plot, bbox_inches='tight')
        plt.close()
        print(f"   -> Grafica RMSE guardada\n")

    except Exception as e:
        print(f"[ERROR CRÍTICO] en {target}: {str(e)}\n")


--- GENERANDO REPORTES VISUALES LINEALES (SIN FUNCIONES) ---

Procesando: Seco - TMAX...
   -> Tabla guardada
   -> Gráfica RMSE guardada

Procesando: Seco - TMIN...
   -> Tabla guardada
   -> Gráfica RMSE guardada

Procesando: Seco - PRECIP...
   -> Tabla guardada
   -> Gráfica RMSE guardada

Procesando: Seco - EVAP...
   -> Tabla guardada
   -> Gráfica RMSE guardada

Procesando: Tropical - TMAX...
   -> Tabla guardada
   -> Gráfica RMSE guardada

Procesando: Tropical - TMIN...
   -> Tabla guardada
   -> Gráfica RMSE guardada

Procesando: Tropical - PRECIP...
   -> Tabla guardada
   -> Gráfica RMSE guardada

Procesando: Tropical - EVAP...
   -> Tabla guardada
   -> Gráfica RMSE guardada

Procesando: Templado - TMAX...
   -> Tabla guardada
   -> Gráfica RMSE guardada

Procesando: Templado - TMIN...
   -> Tabla guardada
   -> Gráfica RMSE guardada

Procesando: Templado - PRECIP...
   -> Tabla guardada
   -> Gráfica RMSE guardada

Procesando: Templado - EVAP...
   -> Tabla guardada
   ->

### Medir Precision con Validation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os
import warnings

warnings.simplefilter('ignore')

bins = [0, 3, 7, 31, 180, 365, 730, float('inf')]
labels = ['1-3 Dias', '4-7 Dias', '8-31 Dias', '32-180 Dias', '181-365 Dias', '1-2 Años', '>2 Años']

validation_route = {
    'Seco': {
        'NR_25037': {'validation': './Sarimax_dataset_splits/Seco/NR_25037/validation/validation_completo.csv',
                     'models':['./models/trained_models/SARIMAX/SARIMAX_25037.0_Seco_EVAP.pkl',
                               './models/trained_models/SARIMAX/SARIMAX_25037.0_Seco_PRECIP.pkl',
                               './models/trained_models/SARIMAX/SARIMAX_25037.0_Seco_TMAX.pkl',
                               './models/trained_models/SARIMAX/SARIMAX_25037.0_Seco_TMIN.pkl']}
    },
    'Templado': {
        'NR_25033': {'validation': './Sarimax_dataset_splits/Templado/NR_25033/validation/validation_completo.csv',
                     'models':['./models/trained_models/SARIMAX/SARIMAX_25033.0_Templado_EVAP.pkl',
                               './models/trained_models/SARIMAX/SARIMAX_25033.0_Templado_PRECIP.pkl',
                               './models/trained_models/SARIMAX/SARIMAX_25033.0_Templado_TMAX.pkl',
                               './models/trained_models/SARIMAX/SARIMAX_25033.0_Templado_TMIN.pkl']}
    },
    'Tropical': {
        'NR_25046': {'validation': './Sarimax_dataset_splits/Tropical/NR_25046/validation/validation_completo.csv',
                     'models':['./models/trained_models/SARIMAX/SARIMAX_25046.0_Tropical_EVAP.pkl',
                               './models/trained_models/SARIMAX/SARIMAX_25046.0_Tropical_PRECIP.pkl',
                               './models/trained_models/SARIMAX/SARIMAX_25046.0_Tropical_TMAX.pkl',
                               './models/trained_models/SARIMAX/SARIMAX_25046.0_Tropical_TMIN.pkl']}
    }
}

print("--- INICIANDO EVALUACIÓN VISUAL SARIMAX (VALIDACIÓN) ---\n")

for region, stations in validation_route.items():
    for station_key, data in stations.items():
        val_path = data['validation']
        model_list = data['models']
        
        print(f"--> Cargando datos para: {region} - {station_key}")
        
        if not os.path.exists(val_path):
            print(f"    [ERROR] No existe el archivo: {val_path}")
            continue
            
        try:
            df_raw = pd.read_csv(val_path, index_col=0, parse_dates=True).asfreq('D')
            df_raw = df_raw.apply(pd.to_numeric, errors='coerce') 
            
            # Relleno simple solo para alimentar el modelo (el modelo necesita continuidad)
            df_filled = df_raw.copy().ffill().bfill()
            
        except Exception as e:
            print(f"    [ERROR] Al leer CSV: {e}")
            continue

        for model_path in model_list:
            if not os.path.exists(model_path):
                continue
            
            filename = os.path.basename(model_path)
            target = filename.split('_')[-1].replace('.pkl', '')
            
            if target not in df_filled.columns:
                continue

            print(f"    Evaluando modelo: {target}...")
            
            # Carpeta de salida
            output_route = f"./data_analysis/Performance_metrics_plots/SARIMAX_VAL/{region}/{station_key}/{target}/"
            os.makedirs(output_route, exist_ok=True)

            try:
                # 1. Cargar Modelo
                model_fit = joblib.load(model_path)
                df_run = df_filled.copy()
                # 2. Preparar Exogenas incluyendo Fourier
                exog_needed = [x for x in model_fit.model.exog_names if x != 'const']
                if any('sin_' in col for col in exog_needed):
                    day_of_year = df_run.index.dayofyear
                    for k in [1, 2]:
                        df_run[f'sin_{k}'] = np.sin(2 * np.pi * k * day_of_year / 365.25)
                        df_run[f'cos_{k}'] = np.cos(2 * np.pi * k * day_of_year / 365.25)
                
                if len(exog_needed) > 0:
                    # Asegurar que existan todas las columnas exógenas
                    for c in exog_needed:
                        if c not in df_run.columns: df_run[c] = 0
                    X_exog = df_run[exog_needed]
                else:
                    X_exog = None
                
                # 3. Aplicar modelo (Validation / Forecast one-step ahead)
                # 'apply' extiende el filtro de Kalman a los nuevos datos sin reentrenar parámetros
                new_results = model_fit.apply(df_run[target], exog=X_exog)
                y_pred_values = new_results.fittedvalues
                
                # 4. Crear DataFrame de Evaluación
                df_eval = pd.DataFrame({
                    'y_real': df_raw[target], # Usamos el RAW (con huecos reales si los hay) como verdad
                    'y_pred': y_pred_values
                })
                
                # Generar Horizontes
                df_eval['horizon_days'] = np.arange(1, len(df_eval) + 1)
                df_eval['range'] = pd.cut(df_eval['horizon_days'], bins=bins, labels=labels)

                # Guardar Predicciones
                csv_pred_path = os.path.join(output_route, f"predictions_val_{target}.csv")
                df_eval.to_csv(csv_pred_path)
                
                # Filtrar para métricas (donde exista dato real)
                df_eval_valid = df_eval.dropna(subset=['y_real'])

                # 5. Calcular Métricas por Horizonte
                metrics_list = []
                for label in labels:
                    subset = df_eval_valid[df_eval_valid['range'] == label]
                    
                    if len(subset) > 0:
                        y_true = subset['y_real'].values
                        y_pred = subset['y_pred'].values
                        
                        mse = np.mean((y_true - y_pred) ** 2)
                        rmse = np.sqrt(mse)
                        mae = np.mean(np.abs(y_true - y_pred))
                        
                        mask_nonzero = y_true != 0
                        if np.sum(mask_nonzero) > 0:
                            mape = np.mean(np.abs((y_true[mask_nonzero] - y_pred[mask_nonzero]) / y_true[mask_nonzero]))
                        else:
                            mape = 0.0
                        count = len(y_true)
                    else:
                        mse, rmse, mae, mape, count = 0, 0, 0, 0, 0
                    
                    metrics_list.append({'range': label, 'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'Count': count})
                
                df_grouped_metrics = pd.DataFrame(metrics_list)

                # Guardar CSV Resumen
                csv_metrics_path = os.path.join(output_route, f"metrics_summary_{target}.csv")
                df_grouped_metrics.to_csv(csv_metrics_path, index=False)

                # 6. Generar Gráficos
                
                # A. Tabla Visual de Métricas
                display_df = df_grouped_metrics.copy()
                for c in ['MSE', 'RMSE', 'MAE', 'MAPE']:
                    display_df[c] = display_df[c].apply(lambda x: f"{x:.4f}" if pd.notnull(x) else "-")
                
                rows, cols = display_df.shape
                fig, ax = plt.subplots(figsize=(max(8, min(24, 1.5 * cols)), max(2, 0.5 * rows)))
                ax.axis('off')
                table = ax.table(cellText=display_df.values, colLabels=display_df.columns, loc='center', cellLoc='center')
                table.auto_set_font_size(False)
                table.set_fontsize(10)
                table.scale(1, 1.5)
                plt.title(f"Métricas Validación - {target}", y=1.1)
                plt.tight_layout()
                plt.savefig(os.path.join(output_route, "metrics_table_VAL.png"), dpi=150, bbox_inches='tight')
                plt.close()

                # B. Gráfico de Barras (RMSE por Horizonte)
                plt.figure(figsize=(10, 6))
                plot_data = df_grouped_metrics[df_grouped_metrics['Count'] > 0]
                if not plot_data.empty:
                    bars = plt.bar(plot_data['range'], plot_data['RMSE'], color='skyblue', edgecolor='black')
                    plt.title(f'Evolución del Error (RMSE) - {region} - {target}')
                    plt.xlabel('Horizonte de Tiempo')
                    plt.ylabel('RMSE')
                    plt.xticks(rotation=45)
                    # Etiquetas sobre las barras
                    for bar in bars:
                        height = bar.get_height()
                        plt.text(bar.get_x() + bar.get_width()/2., height, f'{height:.2f}', ha='center', va='bottom')
                else:
                    plt.text(0.5, 0.5, 'Sin datos válidos', ha='center')
                    
                plt.savefig(os.path.join(output_route, "RMSE_bar_plot_VAL.png"), bbox_inches='tight')
                plt.close()
                
                # C. Gráfico de Línea (Real vs Predicho) - NUEVO (Igual que TensorFlow)
                plt.figure(figsize=(12, 5))
                # Graficar primeros 150 días o todo si es menos
                limit = 150
                subset_plot = df_eval_valid.head(limit)
                
                if not subset_plot.empty:
                    plt.plot(subset_plot.index, subset_plot['y_real'], label='Real', color='blue', alpha=0.6)
                    plt.plot(subset_plot.index, subset_plot['y_pred'], label='Predicción SARIMAX', color='orange', linestyle='--', alpha=0.8)
                    plt.title(f'Predicción vs Realidad (Primeros {limit} días) - {target}')
                    plt.legend()
                    plt.grid(True, alpha=0.3)
                    plt.xlabel('Fecha')
                    plt.ylabel(target)
                else:
                    plt.text(0.5, 0.5, 'Sin datos para graficar', ha='center')

                plt.savefig(os.path.join(output_route, f"line_plot_comparison_VAL.png"), bbox_inches='tight')
                plt.close()
                
                print(f"      -> OK. Reportes guardados en {output_route}")

            except Exception as e:
                print(f"      [ERROR CRÍTICO] Evaluando {target}: {e}")

print("\n--- PROCESO TERMINADO ---")

--- INICIANDO EVALUACIÓN VISUAL SARIMAX (VALIDACIÓN) ---

--> Cargando datos para: Seco - NR_25037
    Evaluando modelo: EVAP...
      -> OK. Reportes guardados en ./data_analysis/Performance_metrics_plots/SARIMAX_VAL/Seco/NR_25037/EVAP/
    Evaluando modelo: PRECIP...
      -> OK. Reportes guardados en ./data_analysis/Performance_metrics_plots/SARIMAX_VAL/Seco/NR_25037/PRECIP/
    Evaluando modelo: TMAX...
      -> OK. Reportes guardados en ./data_analysis/Performance_metrics_plots/SARIMAX_VAL/Seco/NR_25037/TMAX/
    Evaluando modelo: TMIN...
      -> OK. Reportes guardados en ./data_analysis/Performance_metrics_plots/SARIMAX_VAL/Seco/NR_25037/TMIN/
--> Cargando datos para: Templado - NR_25033
    Evaluando modelo: EVAP...
      -> OK. Reportes guardados en ./data_analysis/Performance_metrics_plots/SARIMAX_VAL/Templado/NR_25033/EVAP/
    Evaluando modelo: PRECIP...
      -> OK. Reportes guardados en ./data_analysis/Performance_metrics_plots/SARIMAX_VAL/Templado/NR_25033/PRECIP/
    Ev